# Evaluating essentia key extractor over Beatport EDM datasets.

In this colabolatory notebook we are going going to show how to load Beatport EDM tempo dataset. And we are going to evalute the essentia key extractor `KeyExtractor(...)`.


### Importing mir_eval

In [ ]:
!pip install git+https://github.com/PRamoneda/mir_eval.git@Pedro/only_key
import mir_eval


  Cloning https://github.com/PRamoneda/mir_eval.git (to revision Pedro/only_key) to /tmp/pip-req-build-mcolnv_y
  Running command git clone -q https://github.com/PRamoneda/mir_eval.git /tmp/pip-req-build-mcolnv_y
  Running command git checkout -b Pedro/only_key --track origin/Pedro/only_key
  Switched to a new branch 'Pedro/only_key'
  Branch 'Pedro/only_key' set up to track remote branch 'Pedro/only_key' from 'origin'.
  Created wheel for mir-eval: filename=mir_eval-0.6-cp36-none-any.whl size=97582 sha256=f3ca4c815374ff47866dbccbde71ab0b1580fb75a65ea753b917ec305e380680
  Stored in directory: /tmp/pip-ephem-wheel-cache-k_6vipib/wheels/0e/26/dd/6dc42f9f128108b06932044ac1f78f18635e85593f59bfefc5
Successfully built mir-eval


### Importing mirdata and loading Beatport key dataset


In [ ]:
!pip install git+https://github.com/mir-dataset-loaders/mirdata.git@Pedro/bug/beatport_key

  Cloning https://github.com/mir-dataset-loaders/mirdata.git (to revision Pedro/bug/beatport_key) to /tmp/pip-req-build-srl4pso6
  Running command git clone -q https://github.com/mir-dataset-loaders/mirdata.git /tmp/pip-req-build-srl4pso6
  Running command git checkout -b Pedro/bug/beatport_key --track origin/Pedro/bug/beatport_key
  Switched to a new branch 'Pedro/bug/beatport_key'
  Branch 'Pedro/bug/beatport_key' set up to track remote branch 'Pedro/bug/beatport_key' from 'origin'.
     |████████████████████████████████| 184kB 3.4MB/s 
     |████████████████████████████████| 61kB 4.8MB/s 
     |████████████████████████████████| 5.6MB 9.0MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
  Created wheel for mirdata: filename=mirdata-0.2.0-cp36-none-any.whl size=2361900 sha256=6ed2fd4e6b9250abfd00bd5746fdb9fe016e00986f55c79859dbf18d637fff1d
  Stored in directory: /tmp/p

In [ ]:
import mirdata.beatport_key as bp_key

bp_key.download()

0.00B [00:00, ?B/s]

Starting to download ['keys', 'metadata', 'audio'] to folder /root/mir_datasets/beatport_key
> downloading keys


376kB [00:01, 240kB/s]                            
0.00B [00:00, ?B/s]

> downloading metadata


904kB [00:01, 538kB/s]                            
0.00B [00:00, ?B/s]

> downloading audio


1.98GB [05:06, 6.95MB/s]                            


In [ ]:
bp_key.validate()

Success: the dataset is complete and all files are valid.
--------------------


({}, {})

### Importing essentia

In [ ]:
!pip install essentia


     |████████████████████████████████| 11.7MB 275kB/s 


In [ ]:
from essentia.standard import KeyExtractor, MonoLoader


In [ ]:
bp_key_results = {
    'num_tracks': 0,
    '0.0': 0,
    '0.1': 0,
    '0.2': 0,
    '0.3': 0,
    '0.5': 0,
    '1.0': 0
}


In [ ]:

for k, v in bp_key.load().items():
  # compute estimated key
  loader = MonoLoader(filename=v.audio_path)
  audio = loader()
  key_extractor = KeyExtractor()
  estimated_key = key_extractor(audio)
  estimated_key = estimated_key[0] + ' ' + estimated_key[1]
  # copmpute the best score of the possible keys
  score = []
  for r_key in v.key:
    if r_key.split(' ')[0].lower() == 'x': # if is ambiguous
        reference_key = 'X'
    else:
        reference_key = r_key.split(' ')[0] + ' ' + r_key.split(' ')[1]
    # _ and ^ not supported by mir_eval and essentia
    if not '^' in reference_key and not '_' in reference_key:
      score.append(mir_eval.key.weighted_score(reference_key, estimated_key))
  # add to results and print
  if not '^' in reference_key and not '_' in reference_key:
    bp_key_results[str(max(score))] += 1
    bp_key_results['num_tracks'] += 1
  print("Reference key:", v.key, "Estimated key:", estimated_key, score)


Reference key: ['D minor'] Estimated key: D minor [1.0]
Reference key: ['F# minor'] Estimated key: E major [0.0]
Reference key: ['D minor'] Estimated key: D major [0.2]
Reference key: ['G# minor'] Estimated key: F# major [0.0]
Reference key: ['C minor'] Estimated key: C minor [1.0]
Reference key: ['D minor'] Estimated key: A minor [0.5]
Reference key: ['D# minor'] Estimated key: Eb major [0.2]
Reference key: ['E major'] Estimated key: E major [1.0]
Reference key: ['G# minor'] Estimated key: Ab minor [1.0]
Reference key: ['E minor', 'E major'] Estimated key: E major [0.2, 1.0]
Reference key: ['F# minor'] Estimated key: F# minor [1.0]
Reference key: ['C# major'] Estimated key: C# major [1.0]
Reference key: ['B minor'] Estimated key: B minor [1.0]
Reference key: ['D major'] Estimated key: D major [1.0]
Reference key: ['D major'] Estimated key: D major [1.0]
Reference key: ['G# major'] Estimated key: C major [0.0]
Reference key: ['X'] Estimated key: B major [0.0]
Reference key: ['C# minor'

In [ ]:
print("RESULTS:")
print("Same key: " + str(bp_key_results['1.0']/bp_key_results['num_tracks']*100) + "%")
print("Estimated key is a perfect fifth above reference key: " + str(bp_key_results['0.5']/bp_key_results['num_tracks']*100) + "%")
print("Relative major/minor: " + str(bp_key_results['0.3']/bp_key_results['num_tracks']*100) + "%")
print("Parallel major/minor: " + str(bp_key_results['0.2']/bp_key_results['num_tracks']*100) + "%")
print("Other: " + str(bp_key_results['0.0']/bp_key_results['num_tracks']*100) + "%")


RESULTS:
Same key: 53.926009815024535%
Estimated key is a perfect fifth above reference key: 4.095885239713099%
Relative major/minor: 4.435636089090222%
Parallel major/minor: 16.81766704416761%
Other: 20.724801812004532%
